In [ ]:
!pip install -q -U torch torchvision
!pip install -q git+https://github.com/facebookresearch/fvcore.git
import torch, torchvision
torch.__version__

     |████████████████████████████████| 776.3 MB 20 kB/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.12.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.12.0 which is incompatible.
fastai 2.6.3 requires torch<1.12,>=1.7.0, but you have torch 1.12.0 which is incompatible.
     |████████████████████████████████| 596 kB 8.6 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.6.3 requires torch<1.12,>=1.7.0, but you have torch 1.12.0 which is incompatible.


'1.12.0+cu102'

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -q -e detectron2_repo

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 14407, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14407 (delta 5), reused 7 (delta 2), pack-reused 14392
Receiving objects: 100% (14407/14407), 5.90 MiB | 11.70 MiB/s, done.
Resolving deltas: 100% (10421/10421), done.
     |████████████████████████████████| 79 kB 2.3 MB/s 
     |████████████████████████████████| 151 kB 54.5 MB/s 
     |████████████████████████████████| 1.4 MB 51.9 MB/s 
     |████████████████████████████████| 512 kB 15.5 MB/s 
     |████████████████████████████████| 248 kB 58.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 843 kB 61.9 MB/s 
     |████████████████████████████████| 117 kB 72.8 MB/s 
ERROR: Command err

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# download, decompress the data
!cp "/content/drive/MyDrive/DOESULI/TRAIN2.zip" "/content/"
!unzip TRAIN2.zip > /dev/null
!cp "/content/drive/MyDrive/DOESULI/TRAIN2.json" "/content/"
!mv "/content/TRAIN2.json" "/content/content/"

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("pointsources", {}, "./content/TRAIN2.json", "./content/train")

In [ ]:
train_metadata = MetadataCatalog.get("pointsources")
tr_dataset_dicts = DatasetCatalog.get("pointsources")

In [ ]:
# download, decompress the data
!cp "/content/drive/MyDrive/DOESULI/TEST2.zip" "/content/"
!unzip TEST2.zip > /dev/null
!cp "/content/drive/MyDrive/DOESULI/TEST2.json" "/content/"
!mkdir testing
!mv "/content/TEST2.json" "/content/testing/"
!mv "/content/content/test/" "/content/testing/"

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("testsources", {}, "./testing/TEST2.json", "./testing/test")

In [ ]:
test_metadata = MetadataCatalog.get("testsources")
tst_dataset_dicts = DatasetCatalog.get("testsources")

In [ ]:
from detectron2.engine import DefaultTrainer, hooks, launch
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer

import detectron2.utils.comm as comm
import torch
import time
import datetime
import os
import tensorflow as tf

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("pointsources",)
cfg.DATASETS.TEST = ("testsources", )   # no metrics implemented for this dataset
cfg.SEED = -1
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.BASE_LR = 0.02 #base learning rate
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.GAMMA = 0.1
cfg.SOLVER.IMS_PER_BATCH = 2  #batch size
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = (2000, 4000)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 200 #256  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  
cfg.TEST.EVAL_PERIOD = 100

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
trainer.test(cfg, trainer.model, [COCOEvaluator("testsources", ['bbox', 'segm'] , False, cfg.OUTPUT_DIR)])
#trnr = Trainer(cfg)
#trnr.resume_or_load(resume =False)
#trnr.register_hooks(  [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]  )
#NEED TO ADD test_with_TTA 
#trnr.train()
#os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trnr = Trainer.build_model(cfg)
#DetectionCheckpointer(trnr, save_dir=cfg.OUTPUT_DIR).resume_or_load()

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("testsources")
val_loader = build_detection_test_loader(cfg, "testsources")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np


experiment_folder = './output/'

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr(experiment_folder + 'metrics.json')
df = pd.DataFrame(experiment_metrics)
TL = df[0:250]

it = 0
ref_it = TL['iteration']
it_points = []
valid_points = []
for x in df['validation_loss']:
  if not math.isnan(x):
    valid_points.append(x)
    it_points.append(ref_it[it])
  it +=1

fig = plt.figure(figsize=(8,6))
plt.plot(it_points, valid_points, '-o', label="Validation Loss")
plt.plot(TL['iteration'], TL['total_loss'], label="Training Loss")
plt.xlabel("Iteration", fontsize=18)
plt.ylabel("Loss", fontsize=18)
plt.title("Iteration vs. Loss, Set 2", fontsize=22)
plt.legend()

In [ ]:
!zip -r /content/MODEL2OUTPUT.zip /content/output/

In [ ]:
!cp "/content/MODEL2OUTPUT.zip" "/content/drive/MyDrive/DOESULI"

## Benchmark inference speed

In [ ]:
import time
times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

# REAL IMAGE INFERENCE

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set the testing threshold for this model
cfg.DATASETS.TEST = ("inf_rl", )
predictor = DefaultPredictor(cfg)

In [ ]:
!mkdir realimgs
!unzip REALIMG.zip -d /content/realimgs/
!mv /content/realimgs/content/real/ /content/realimgs/
!rm -r /content/realimgs/content/
!mv /content/realimgs/real/ /content/
!rm -r /content/realimgs/

In [ ]:
register_coco_instances("inf_rl", {}, "./real/REALIMG.json", "./real/images")
rl_metadata = MetadataCatalog.get("inf_rl")
rl_dataset_dicts = DatasetCatalog.get("inf_rl")

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(rl_dataset_dicts, 1):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=rl_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

# EXAMPLES FROM TEST AND TRAIN

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set the testing threshold for this model
cfg.DATASETS.TEST = ("testsources", )
predictor = DefaultPredictor(cfg)

In [ ]:
import random

for d in random.sample(tr_dataset_dicts , 3):
  img = cv2.imread(d["file_name"])
  print(d["file_name"])
  visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
  vis = visualizer.draw_dataset_dict(d)
  cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(tst_dataset_dicts, 7): 
    print(d["file_name"])   
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=test_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
import random

for d in random.sample(tst_dataset_dicts , 3):
  img = cv2.imread(d["file_name"])
  visualizer = Visualizer(img[:, :, ::-1], metadata=test_metadata, scale=0.5)
  vis = visualizer.draw_dataset_dict(d)
  cv2_imshow(vis.get_image()[:, :, ::-1])